In [1]:
#Basics 
import numpy as np
import pandas as pd 
import math, random
import sys, os 
import sklearn
from sklearn.metrics import balanced_accuracy_score, RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from scipy import stats 
import scipy
import matplotlib 
import matplotlib.pyplot as plt 
from glob import glob
from itertools import cycle
from joblib import dump, load
%matplotlib inline

#Google Drive setup 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

#Navigate to directory with Google Drive
%cd /content/drive/Shareddrives/CS\ 272\ PCOS\ FL/Model\ Evaluation/

/content/drive/Shareddrives/CS 272 PCOS FL/Model Evaluation


In [3]:
with np.load('best_fl_models/fedprox4_1000.npz') as data:
  # for key in data.files:
  #   print(key)
  fedprox_coefs = data['arr_0']
  fedprox_ints = data['arr_1']
fedprox = LogisticRegression(warm_start=True).fit(np.random.uniform(size=(10,13)), np.arange(10))
fedprox.coef_ = fedprox_coefs
fedprox.intercept_ = fedprox_ints

with np.load('best_fl_models/fedavgm5_750.npz') as data:
  # for key in data.files:
  #   print(key)
  fedavgm_coefs = data['arr_0']
  fedavgm_ints = data['arr_1']
fedavgm = LogisticRegression(warm_start=True).fit(np.random.uniform(size=(10,13)), np.arange(10))
fedavgm.coef_ = fedavgm_coefs
fedavgm.intercept_ = fedavgm_ints

with np.load('best_fl_models/fedavg3_850.npz') as data:
  # for key in data.files:
  #   print(key)
  fedavg_coefs = data['arr_0']
  fedavg_ints = data['arr_1']
fedavg = LogisticRegression(warm_start=True).fit(np.random.uniform(size=(10,13)), np.arange(10))
fedavg.coef_ = fedavg_coefs
fedavg.intercept_ = fedavg_ints

baseline = load('best_fl_models/logreg_baseline.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
#preprocess real test clinic data 
for i in range(10):
  clinic = pd.read_csv(f'../Synthetic Data Generation /clinic_datasets_OOD_challenge_testing/clinic_{i}.csv')
  clinic['response_type'] = clinic.response_type.values - 1
  clinic['treatment'] = clinic.treatment.values - 1
  test = clinic.drop(columns=['response_type', 'treatment'])
  for model, model_name in zip((fedprox, fedavgm, fedavg, baseline), ('FedProx', 'FedAvgM', 'FedAvg', 'Baseline Aggregate Model')):
    preds = model.predict_proba(test.to_numpy())
    clinic['pred_treatment'] = np.argmax(preds, axis=1)
    clinic[[f'{j}_prob' for j in range(10)]] = preds
    clinic.to_csv(f'OOD_challenge_test_clinics/{model_name}/test_clinic_{i}.csv', index=False)
  # fedprox_preds = fedprox.predict_proba(test)
  # fedavgm_preds = fedavgm.predict_proba(test)
  # fedavg_preds = fedavg.predict_proba(test)

In [5]:
# !sudo rm true_test_clinics/baseline_aggr_logreg/*.npy